In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import keras
import tensorflow as tf
from keras import backend as K

print("Keras Version:", keras.__version__)
print("Tensorflow Version:", tf.__version__)
print("image dim ordering:", K.image_dim_ordering())

Using TensorFlow backend.


Keras Version: 2.1.6
Tensorflow Version: 1.9.0
image dim ordering: tf


In [5]:
import _pickle as cPickle
import numpy as np
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
# from tankbuster.cnn import CNNArchitecture

In [16]:
input_dir = "data_reduced/"  # Input directory
learning_rate = 0.001  # Learning rate
opt = "SGD"  # Optimizer
target_size = (150, 150)  # Target size for data augmentation

input_shape = (150, 150,3)
# Configure a callback to reduce the learning rate upon plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=50,
                              cooldown=50, min_lr=0.0001, verbose=1)

# Path to pre-trained weights file, if used. Otherwise None.
weights = None

# Configure the validation parameters
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [30]:
scores = []

# Read data & labels
data, labels, classes = [], [], {}

for (root, subdirs, files) in os.walk(input_dir):
    # Assign a numerical identifier to each class directory
    for i, class_dir in enumerate(subdirs):
        classes[class_dir] = i

    # Define allowed image extensions
    ext = ['png', 'jpg', 'jpeg']

    # Loop over the files in each directory
    for f in files:
        if f.split('.')[-1] in ext:  # Check file extension
            path = os.path.join(root, f)  # Get image path
            label = path.split('/')[-2]  # Extract class label from path
            numlabel = classes[label]  # Get numerical label from the dict

#             print "Now processing {}/{}/{}".format(path,label, numlabel)

            # Load and preprocess image
            image = load_img(path, target_size=target_size)  # Load image
            features = img_to_array(image)  # Convert image to numpy array

            labels.append(numlabel)  # Append label to list
            data.append(features) # Append features to list

In [31]:
# Convert data and labels to numpy arrays
data = np.asarray(data, dtype=np.float32)
labels = np.asarray(labels, dtype=np.float32)

In [32]:
labels.shape

(740,)

In [33]:
data.shape

(740, 150, 150, 3)

In [34]:
from keras.applications import ResNet50
from keras.layers import Flatten, Input, Dense, Dropout, Conv2D,MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam

In [35]:
 for n, (train, test) in enumerate(kfold.split(data, labels)):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(2, activation=tf.nn.softmax))
        
        adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)

        model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
        
        
        # Split the data into training and testing sets
        traindata, trainlabels = data[train], labels[train]
        testdata, testlabels = data[test], labels[test]

        # Convert integer labels into one-hot encoded vectors
        trainlabels = np_utils.to_categorical(trainlabels, 2)
        testlabels = np_utils.to_categorical(testlabels, 2)
        
        
        training_generator = ImageDataGenerator(rescale=1./255)

        # Generate training data
        training_data = training_generator.flow(traindata, trainlabels, batch_size=8)
    
        # Set up generator for validation data
        validation_generator = ImageDataGenerator(rescale=1./255 )

        # Generate validation data
        validation_data = validation_generator.flow(testdata,testlabels,batch_size=8)

        # Start training the model
        training = model.fit_generator(training_data,
                                       steps_per_epoch=256,
                                       epochs=10,
                                       validation_data=validation_data,
                                       validation_steps=256,
                                       callbacks=[reduce_lr])

        # Evaluate the model
        (loss, accuracy) = model.evaluate(testdata,
                                          testlabels,
                                          batch_size=8,
                                          verbose=1)

        # Save weights for each fold into a file
#         model.save_weights('{}-cv-fold_{}.h5'.format(arch, (n + 1)),
#                            overwrite=True)

        # Write the training history for each fold into a file
#         with open('{}-history-fold_{}.pkl'.format(arch, (n + 1)), 'wb') \
#                 as histfile:
#             pickle.dump(training.history, histfile)

        # Append the accuracy to the list of scores
        scores.append(accuracy)

Epoch 1/10
256/256 [==============================] - 65s 255ms/step - loss: 0.6132 - acc: 0.6382 - val_loss: 0.6018 - val_acc: 0.6261
Epoch 2/10
256/256 [==============================] - 62s 241ms/step - loss: 0.5248 - acc: 0.7676 - val_loss: 0.4999 - val_acc: 0.7441
Epoch 3/10
256/256 [==============================] - 62s 243ms/step - loss: 0.4680 - acc: 0.7891 - val_loss: 0.4958 - val_acc: 0.7462
Epoch 4/10
256/256 [==============================] - 64s 249ms/step - loss: 0.4329 - acc: 0.8130 - val_loss: 0.4854 - val_acc: 0.7457
Epoch 5/10
256/256 [==============================] - 65s 254ms/step - loss: 0.4216 - acc: 0.8223 - val_loss: 0.4787 - val_acc: 0.7733
Epoch 6/10
256/256 [==============================] - 62s 241ms/step - loss: 0.4094 - acc: 0.8247 - val_loss: 0.4738 - val_acc: 0.7592
Epoch 7/10
256/256 [==============================] - 63s 246ms/step - loss: 0.4025 - acc: 0.8247 - val_loss: 0.4706 - val_acc: 0.7889
Epoch 8/10
256/256 [==============================] - 6

256/256 [==============================] - 63s 246ms/step - loss: 0.3914 - acc: 0.8237 - val_loss: 0.3997 - val_acc: 0.8114
Epoch 10/10
74/74 [==============================] - 1s 7ms/step
Epoch 1/10
256/256 [==============================] - 66s 256ms/step - loss: 0.6091 - acc: 0.6310 - val_loss: 0.5749 - val_acc: 0.7261
Epoch 2/10
256/256 [==============================] - 62s 243ms/step - loss: 0.5165 - acc: 0.7687 - val_loss: 0.4750 - val_acc: 0.7960
Epoch 3/10
256/256 [==============================] - 62s 243ms/step - loss: 0.4554 - acc: 0.8019 - val_loss: 0.4509 - val_acc: 0.7939
Epoch 4/10
256/256 [==============================] - 63s 245ms/step - loss: 0.4319 - acc: 0.8135 - val_loss: 0.4349 - val_acc: 0.8078
Epoch 5/10
256/256 [==============================] - 63s 245ms/step - loss: 0.4151 - acc: 0.8161 - val_loss: 0.4293 - val_acc: 0.7950
Epoch 6/10
256/256 [==============================] - 63s 244ms/step - loss: 0.3943 - acc: 0.8239 - val_loss: 0.4475 - val_acc: 0.7944
E

In [58]:
# Print the scores and the best fold
print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores), np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))

Scores Mean: 0.7202 and (STDEV 0.0310)
Best result for fold 2
Best accuracy is 0.76


In [40]:
scores

[0.6933333333333334,
 0.72,
 0.76,
 0.72,
 0.7432432432432432,
 0.7297297297297297,
 0.7397260273972602,
 0.684931506849315,
 0.6575342465753424,
 0.7534246575342466]